In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
np.arange(-10, 1)

In [ ]:
from sklearn.model_selection import train_test_split

index = pd.date_range("2021-01-01", periods=7, freq="h")
X = pd.DataFrame(
    index=index,
    data={
        "rain": range(
            -3,
            4,
        )
    },
)
y = pd.Series(
    index=index,
    data=range(
        -3,
        4,
    ),
    name="load",
)
y *= 10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=3, shuffle=False)
X_train

In [ ]:
X_test

In [ ]:
from probafcst.utils.tabularization import create_lagged_features

X_lagged, y_lagged = create_lagged_features(
    X_train, y_train, lags=[1, 2], include_seasonal_dummies=False, is_training=True
)
X_lagged

In [ ]:
y_lagged

In [ ]:
y_pred = y_test.copy()
y_pred.iloc[:] = np.nan

In [ ]:
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_pred])
X_full

In [ ]:
y_full

In [ ]:
y_full

In [ ]:
X_lagged

In [ ]:
X_lagged_test, _ = create_lagged_features(
    X_full, y_full, lags=[1, 2], include_seasonal_dummies=False, is_training=False
)
X_lagged_test

In [ ]:
y_hat = [10, 20, 30]
for timestamp, y_hat in zip(X_test.index, y_hat):
    X_lagged_test, _ = create_lagged_features(
        X_full, y_full, lags=[1, 2], include_seasonal_dummies=False, is_training=False
    )
    X_step = X_lagged_test.loc[[timestamp]]
    y_full.loc[timestamp] = y_hat

In [ ]:
y_train

In [ ]:
from probafcst.models.xgboost import XGBQuantileForecaster

quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]
model = XGBQuantileForecaster(
    lags=[1, 2],
    quantiles=quantiles,
    cyclical_encodings=False,
    include_seasonal_dummies=False,
)
model.fit(y_train, X_train)

In [ ]:
model.feature_names_in_

In [ ]:
X_test

In [ ]:
fh = np.arange(1, len(X_test) + 1)
model.predict_quantiles(fh, X_test, alpha=quantiles)

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
y = pd.read_parquet("../data/bikes.parquet")

X = pd.DataFrame(index=y.index, data=np.random.randn(len(y), 1), columns=["rain"])
X

In [ ]:
y

In [ ]:
from probafcst.utils.tabularization import create_lagged_features

features, labels = create_lagged_features(
    X=None,
    y=y["bike_count"],
    lags=[1, 2],
    X_lag_cols=[],
    include_seasonal_dummies=True,
    is_training=True,
)
features

In [ ]:
features.info()

In [ ]:
from sktime.forecasting.base import ForecastingHorizon

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=30, shuffle=False)
model = XGBQuantileForecaster(
    lags=[1, 7, 30],
    quantiles=quantiles,
    include_seasonal_dummies=True,
    cyclical_encodings=True,
)
model.fit(y_train, X_train)
model

In [ ]:
model.feature_names_in_

In [ ]:
fh = ForecastingHorizon(y_test.index, is_relative=False)
y_pred = model.predict_quantiles(fh, X_test, alpha=quantiles)
y_pred

In [ ]:
import matplotlib.pyplot as plt

plt.barh(model.feature_names_in_, model.model.feature_importances_)

In [ ]:
from probafcst.plotting import plot_quantiles

plot_quantiles(y_test, y_pred)